# Import necessary modules

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch
from collections import Counter
from tensorflow.keras.callbacks import TensorBoard

from sklearn.metrics import roc_auc_score, ndcg_score
from sklearn.preprocessing import LabelBinarizer

import os
os.environ['CRYPTOGRAPHY_OPENSSL_NO_LEGACY'] = '1'


2023-11-28 10:25:09.041805: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 10:25:09.041912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 10:25:09.049195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 10:25:59.350215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Manual pre-processing of data
### behaviors.tsv

#### From documentation:  
The behaviors.tsv file contains the impression logs and users' news click histories. It has 5 columns divided by the tab symbol:
- Impression ID. The ID of an impression.  
- User ID. The anonymous ID of a user.  
- Time. The impression time with format "MM/DD/YYYY HH:MM:SS AM/PM".  
- History. The news click history (ID list of clicked news) of this user before this impression. The clicked news articles are ordered by time.  
- Impressions. List of news displayed in this impression and user's click behaviors on them (1 for click and 0 for non-click). The orders of news in a impressions have been shuffled.  

In [2]:
raw_behaviour = pd.read_csv(
    "behaviors.tsv", 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset consist of {len(raw_behaviour)} number of interactions.")
raw_behaviour.head()

The dataset consist of 156965 number of interactions.


,impressionId,userId,timestamp,click_history,impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [3]:
## Indexize users
unique_userIds = raw_behaviour['userId'].unique()
# Allocate a unique index for each user, but let the zeroth index be a UNK index:
ind2user = {idx +1: itemid for idx, itemid in enumerate(unique_userIds)}
user2ind = {itemid : idx for idx, itemid in ind2user.items()}
print(f"We have {len(user2ind)} unique users in the dataset")

# Create a new column with userIdx:
raw_behaviour['userIdx'] = raw_behaviour['userId'].map(lambda x: user2ind.get(x,0))

We have 50000 unique users in the dataset


## Load article data
We also need to get the content information of each article. We will use the news.tsv file to index the items.

In [4]:
news = pd.read_csv(
    "news.tsv", 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
news.head(2)
# Build index of items
ind2item = {idx +1: itemid for idx, itemid in enumerate(news['itemId'].values)}
item2ind = {itemid : idx for idx, itemid in ind2item.items()}

news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


Now we need to process the click history and impressions. We need to both indexize the strings, but also to decode impressions into clicks and non-clicks.

In [5]:
# Indexize click history field
def process_click_history(s):
    list_of_strings = str(s).split(" ")
    return [item2ind.get(l, 0) for l in list_of_strings]
        
raw_behaviour['click_history_idx'] = raw_behaviour.click_history.map(lambda s:  process_click_history(s))
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,click_history_idx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071..."
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559..."
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[23643, 4853, 27686, 31189]"


In [6]:
# collect one click and one no-click from impressions:
def process_impression(s):
    list_of_strings = s.split(" ")
    itemid_rel_tuple = [l.split("-") for l in list_of_strings]
    noclicks = []
    for entry in itemid_rel_tuple:
        if entry[1] =='0':
            noclicks.append(entry[0])
        if entry[1] =='1':
            click = entry[0]
    return noclicks, click

raw_behaviour['noclicks'], raw_behaviour['click'] = zip(*raw_behaviour['impressions'].map(process_impression))
# We can then indexize these two new columns:
raw_behaviour['noclicks'] = raw_behaviour['noclicks'].map(lambda list_of_strings: [item2ind.get(l, 0) for l in list_of_strings])
raw_behaviour['click'] = raw_behaviour['click'].map(lambda x: item2ind.get(x,0))

raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,click_history_idx,noclicks,click
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",[50689],33900
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...","[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...","[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...","[50689, 50106, 50022]",50715
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[23643, 4853, 27686, 31189]","[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475


In [7]:
# convert timestamp value to hours since epoch
raw_behaviour['epochhrs'] = pd.to_datetime(raw_behaviour['timestamp']).values.astype(np.int64)/(1e6)/1000/3600
raw_behaviour['epochhrs'] = raw_behaviour['epochhrs'].round()
raw_behaviour[['click','epochhrs']].groupby("click").min("epochhrs").reset_index()

,click,epochhrs
0,13,437047.0
1,28,437100.0
2,48,437079.0
3,75,437069.0
4,86,437150.0
...,...,...
6346,51263,437079.0
6347,51269,437063.0
6348,51271,437069.0
6349,51275,437063.0


In [8]:
raw_behaviour

,impressionId,userId,timestamp,click_history,impressions,userIdx,click_history_idx,noclicks,click,epochhrs
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",[50689],33900,437073.0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...","[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187,437106.0
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...","[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767,437143.0
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...","[50689, 50106, 50022]",50715,437069.0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[23643, 4853, 27686, 31189]","[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475,437104.0
...,...,...,...,...,...,...,...,...,...,...
156960,156961,U21593,11/14/2019 10:24:05 PM,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...,7282,"[21178, 19891, 2656, 14202, 16230, 21717, 1626...","[36709, 38174, 39698, 31886, 39748, 42502, 383...",39852,437158.0
156961,156962,U10123,11/13/2019 6:57:04 AM,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...,1036,"[27148, 6338, 19239, 6951, 7631, 20288, 8014, ...","[39247, 43708, 47008, 5828, 40641, 34662, 4619...",45322,437119.0
156962,156963,U75630,11/14/2019 10:58:13 AM,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...,4834,"[19161, 26886, 30517, 27148, 22579, 9315, 1336...","[37872, 41089, 36783, 31290, 34314, 38166, 342...",40345,437147.0
156963,156964,U44625,11/13/2019 2:57:02 PM,N4118 N47297 N3164 N43295 N6056 N38747 N42973 ...,N6219-0 N3663-0 N31147-0 N58363-0 N4107-0 N457...,30674,"[11411, 15585, 2261, 3682, 31666, 1132, 16302,...","[47215, 43622, 35835, 33276, 35081, 35780, 443...",42301,437127.0


# Modeling
We want to make a matrix factorization model where each user $u$ has a d-dimensional parameter vector $z_u$ and each item $i$ has a parameter vector $v_i$. This implies that we do not need the click_history_idx column when we train the model (why?).

Second, to simplify the computation of things we assume that the user only considers two items in each interaction: The item the user clicked on, and the first item in the noclicks list (what are we missing by this assumption?).

Hence, our dataset consist of a `behaviour` dataframe and a `news` dataframe with content information of the news articles. We will use a language model to interpret our article data, and for simplicity we will only use the title as the text input here.

In [9]:
raw_behaviour['noclick'] = raw_behaviour['noclicks'].map(lambda x : x[0])
behaviour = raw_behaviour[['epochhrs','userIdx','click_history_idx','noclick','click']]
behaviour.head()

,epochhrs,userIdx,click_history_idx,noclick,click
0,437073.0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",50689,33900
1,437106.0,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...",37405,32187
2,437143.0,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...",39528,5767
3,437069.0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...",50689,50715
4,437104.0,5,"[23643, 4853, 27686, 31189]",2006,31475


In [10]:
# Let us use the last 10pct of the data as our validation data:
test_time_th = behaviour['epochhrs'].quantile(0.9)
train = behaviour[behaviour['epochhrs']< test_time_th]
valid =  behaviour[behaviour['epochhrs']>= test_time_th]

In [11]:
class MindDataset(Dataset):
    # A fairly simple torch dataset module that can take a pandas dataframe (as above), 
    # and convert the relevant fields into a dictionary of arrays that can be used in a dataloader
    def __init__(self, df):
        # Create a dictionary of tensors out of the dataframe
        self.data = {
            'userIdx' : torch.tensor(df.userIdx.values),
            'click' : torch.tensor(df.click.values),
            'noclick' : torch.tensor(df.noclick.values)
        }
    def __len__(self):
        return len(self.data['userIdx'])
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.data.items()}

In [12]:
# Build datasets and dataloaders of train and validation dataframes:
bs = 1024
ds_train = MindDataset(train)
train_loader = DataLoader(ds_train, batch_size=bs, shuffle=True)
ds_valid = MindDataset(valid)
valid_loader = DataLoader(ds_valid, batch_size=bs, shuffle=False)

batch = next(iter(train_loader))

In [13]:
batch["noclick"]

tensor([32483,  5764, 39519,  ..., 20302, 36333, 48698])

## Model

#### Framework
We will use pytorch-lightning to define and train our model. It is a high-level framework (similar to fastAI) but with a slightly different way of defining things. It is my personal go-to framework and is very flexible. For more information, see https://pytorch-lightning.readthedocs.io/.

#### The model
We assume that each interaction goes as follow: the user is presented with two items: the click and no-click item. After the user reviewed both items, she will choose the most relevant one. This can be modeled as a categorical distirbution with two options (yes, you could do binomial). There is a loss function in pytorch for this already, called the `F.cross_entropy` that we will use.

In [14]:
# Build a matrix factorization model
class NewsMF(pl.LightningModule):
    def __init__(self, num_users, num_items, dim = 10):
        super().__init__()
        self.dim=dim
        self.useremb = nn.Embedding(num_embeddings=num_users, embedding_dim=dim)
        self.itememb = nn.Embedding(num_embeddings=num_items, embedding_dim=dim)
    
    def forward(self, user, item):
        batch_size = user.size(0)
        uservec = self.useremb(user)
        itemvec = self.itememb(item)

        score = (uservec*itemvec).sum(-1).unsqueeze(-1)
        
        return score
    
    def training_step(self, batch, batch_idx):
        batch_size = batch['userIdx'].size(0)

        score_click = self.forward(batch['userIdx'], batch['click'])
        score_noclick = self.forward(batch['userIdx'], batch['noclick'])
        
        scores_all = torch.concat((score_click, score_noclick), dim=1)
        # Compute loss as cross entropy (categorical distribution between the clicked and the no clicked item)
        loss = F.cross_entropy(input=scores_all, target=torch.zeros(batch_size, device=scores_all.device).long())
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        # for now, just do the same computation as during training
        loss = self.training_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    

In [15]:
# Build and train the model
# mf_model = NewsMF(num_users=len(ind2user)+1, num_items = len(ind2item)+1)
# trainer = pl.Trainer(max_epochs=10, accelerator="gpu", logger=logger)
# trainer.fit(model=mf_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

# Save the model
# trainer.save_checkpoint("model_user.ckpt")

# Load the trained model
mf_model = NewsMF.load_from_checkpoint(checkpoint_path="model_user.ckpt", num_users=len(ind2user) + 1, num_items=len(ind2item) + 1)

# Sense Check

In [16]:
USER_ID = 2350

# Suggested Items

In [17]:
# Create item_ids and user ids list
item_id = list(ind2item.keys())
userIdx =  [USER_ID]*len(item_id)

preditions = mf_model.forward(torch.IntTensor(userIdx), torch.IntTensor(item_id))

# Select top 10 argmax
top_index = torch.topk(preditions.flatten(), 10).indices

# Filter for top 10 suggested items
filters = [ind2item[ix.item()] for ix in top_index]
news[news["itemId"].isin(filters)]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
822,N31931,news,newscrime,Florida man arrested in beating death of peepi...,Victor Vickery had been in bed with his girlfr...,https://assets.msn.com/labs/mind/AAJ4PC3.html,"[{""Label"": ""Florida Man"", ""Type"": ""U"", ""Wikida...",[]
2461,N23784,music,music-gallery,35 musicians you should see live at least once...,If you have a chance to see these musicians li...,https://assets.msn.com/labs/mind/AAJfXsG.html,[],[]
6466,N62839,news,newscrime,One arrested in connection with September inte...,Tavarious Jones has been arrested and charged ...,https://assets.msn.com/labs/mind/AAJlwTZ.html,[],"[{""Label"": ""Interstate 240 (Tennessee)"", ""Type..."
7454,N42791,tv,tvnews,Eva Longoria: 'Latinas Are the Last to Find Th...,Eva Longoria delivered a powerful and moving k...,https://assets.msn.com/labs/mind/AAJm1R3.html,"[{""Label"": ""Eva Longoria"", ""Type"": ""P"", ""Wikid...","[{""Label"": ""Eva Longoria"", ""Type"": ""P"", ""Wikid..."
10334,N7558,foodanddrink,newstrends,Jean-Georges's Flagship Pulls in $25M a Year B...,"Plus, a famed Hawaii chef is coming to cook in...",https://assets.msn.com/labs/mind/AAIZ6Na.html,"[{""Label"": ""Jean-Georges Vongerichten"", ""Type""...","[{""Label"": ""Hawaii"", ""Type"": ""G"", ""WikidataId""..."
18957,N4693,lifestyle,lifestylecelebstyle,8 Times Ciara Was the Sexiest Person on the Re...,"Ciara's red carpet style is automatic, superso...",https://assets.msn.com/labs/mind/AAJblvp.html,"[{""Label"": ""Ciara"", ""Type"": ""P"", ""WikidataId"":...","[{""Label"": ""Ciara"", ""Type"": ""P"", ""WikidataId"":..."
26445,N32254,sports,baseball_mlb,"Gabe Kapler, days after Phillies fired him, ha...",Gabe Kapler's unemployment may not last too lo...,https://assets.msn.com/labs/mind/AAIGIL2.html,"[{""Label"": ""Gabe Kapler"", ""Type"": ""P"", ""Wikida...","[{""Label"": ""Gabe Kapler"", ""Type"": ""P"", ""Wikida..."
27254,N4229,news,newspolitics,Trump's envoy to testify that 'no quid pro quo...,The U.S. ambassador to the European Union is e...,https://assets.msn.com/labs/mind/AAIGOD4.html,"[{""Label"": ""Donald Trump"", ""Type"": ""P"", ""Wikid...","[{""Label"": ""United States Ambassador to the Eu..."
29162,N62342,video,science,The mysterious case of NASA's missing $1.1 bil...,"In the '60s, NASA commissioned Grumman Aircraf...",https://assets.msn.com/labs/mind/AAIS0DY.html,"[{""Label"": ""NASA"", ""Type"": ""O"", ""WikidataId"": ...","[{""Label"": ""NASA"", ""Type"": ""O"", ""WikidataId"": ..."
30997,N50367,sports,icehockey_nhl,Zdeno Chara Celebrates 'Great Team Win' With I...,Zdeno Chara still can make his presence known ...,https://assets.msn.com/labs/mind/AAJrENn.html,"[{""Label"": ""Zdeno Ch\u00e1ra"", ""Type"": ""P"", ""W...","[{""Label"": ""Zdeno Ch\u00e1ra"", ""Type"": ""P"", ""W..."


# Historical Items

In [18]:
click_ids = behaviour[behaviour["userIdx"]==USER_ID]["click"].values
ll = lambda x: "N"+str(x)

click_ids = [ll(each) for each in click_ids]

news[news["itemId"].isin(click_ids)]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
20810,N33900,sports,football_nfl,Patriots Vs. Jets Live: New England Steamrolls...,Final Patriots - : The New England Patriots im...,https://assets.msn.com/labs/mind/AAJ7aqU.html,"[{""Label"": ""New England Patriots"", ""Type"": ""O""...","[{""Label"": ""New England Patriots"", ""Type"": ""O""..."
28754,N32675,news,newsus,Northbound Texas 130 in Pflugerville closing f...,Northbound Texas 130 at Pflugerville Parkway w...,https://assets.msn.com/labs/mind/AAJpxWP.html,"[{""Label"": ""Texas"", ""Type"": ""G"", ""WikidataId"":...","[{""Label"": ""Texas"", ""Type"": ""G"", ""WikidataId"":..."
33795,N34112,travel,travelarticle,Budweiser Clydesdale to strut stuff at Jackson...,Residents can get an up-close glimpse or selfi...,https://assets.msn.com/labs/mind/BBWLjXu.html,"[{""Label"": ""Winn-Dixie"", ""Type"": ""O"", ""Wikidat...","[{""Label"": ""Winn-Dixie"", ""Type"": ""O"", ""Wikidat..."
33846,N32603,news,newsus,Joke Made About President Trump During North S...,An inappropriate joke made during a North Surr...,https://assets.msn.com/labs/mind/BBWJlkC.html,[],"[{""Label"": ""White House"", ""Type"": ""F"", ""Wikida..."
43516,N39247,news,newsus,Allegheny County Controller Chelsa Wagner Goes...,Allegheny County Controller Chelsa Wagner goes...,https://assets.msn.com/labs/mind/BBWBTtb.html,[],"[{""Label"": ""Detroit"", ""Type"": ""G"", ""WikidataId..."
44747,N45975,video,news,Desperation? Trump's top aide now suing Trump ...,The potential linchpin to the impeachment case...,https://assets.msn.com/labs/mind/BBWBZ1g.html,[],"[{""Label"": ""Ari Melber"", ""Type"": ""P"", ""Wikidat..."
45916,N43459,sports,football_nfl,Will Cain Ranks Three Teams Ahead Of Patriots ...,https://www.dailymotion.com/embed/video/x ns u...,https://assets.msn.com/labs/mind/BBWwsaG.html,"[{""Label"": ""New England Patriots"", ""Type"": ""O""...","[{""Label"": ""New England Patriots"", ""Type"": ""O""..."
